In [1]:
import mysql.connector as mc
conn=mc.connect
(
    host:= 'locahost',
    port:='3306',
    user:='system',
    password:='roots',
    database:='oracle'
    )


('locahost', '3306', 'system', 'roots', 'oracle')

In [1]:
pip install numpy

In [5]:
import numpy as np
arr1=np.array([1,2])
arr1

array([1, 2])

In [7]:
arr1.ndim

1

In [8]:
arr1.size

2

In [9]:
arr1.shape

(2,)

In [12]:
b=np.array(my_m)
b

array([[ 1,  2,  3,  4],
       [ 5,  8,  7,  9],
       [ 6,  7,  4,  2],
       [10,  1, 11, 12]])

In [13]:
b.ndim

2

In [14]:
b.shape

(4, 4)

In [15]:
b.size

16

In [16]:
int8_arr=np.array([1,2,3],dtype=np.int8)
int16_arr=np.array([1000,2000,3000],dtype=np.int16)
int32_arr=np.array([100000,200000,300000],dtype=np.int32)

In [1]:

import json
import os
import math
from datetime import datetime

MOVIES_STATE_FILE = "movies_state.json"
BOOKINGS_FILE = "bookings.json"

DEFAULT_MOVIES = [
    {"id": 1, "name": "Avengers: Endgame", "price": 200.0, "rating": "U", "seats": 50},
    {"id": 2, "name": "Joker", "price": 250.0, "rating": "A", "seats": 30},
    {"id": 3, "name": "Pushpa 2", "price": 300.0, "rating": "UA", "seats": 40},
]

GST_RATE = 0.05
MAX_TICKETS_PER_TX = 6


# --------------------------- Persistence helpers ---------------------------

def load_movies():
    """Load movies state from JSON file; if missing, initialize with defaults."""
    if os.path.exists(MOVIES_STATE_FILE):
        try:
            with open(MOVIES_STATE_FILE, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception:
            # corrupt file -> reinitialize
            pass
    # assign fresh copy of default movies
    movies = DEFAULT_MOVIES.copy()
    save_movies(movies)
    return movies


def save_movies(movies):
    try:
        with open(MOVIES_STATE_FILE, "w", encoding="utf-8") as f:
            json.dump(movies, f, indent=2)
    except Exception as e:
        print("Warning: could not save movies state:", e)


def load_bookings():
    if os.path.exists(BOOKINGS_FILE):
        try:
            with open(BOOKINGS_FILE, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception:
            pass
    return []


def save_booking(booking):
    bookings = load_bookings()
    bookings.append(booking)
    try:
        with open(BOOKINGS_FILE, "w", encoding="utf-8") as f:
            json.dump(bookings, f, indent=2)
    except Exception as e:
        print("Warning: could not save booking:", e)


# --------------------------- Core functions ---------------------------

def show_movies(movies):
    print("\nAvailable Movies:\n")
    print(f{"ID":<4}{"Name":<30}{"Price (₹)":>12}  {"Rating":>8}  {"Seats":>6}")
    print("-" * 64)
    for m in movies:
        print(f"{m['id']:<4}{m['name']:<30}{m['price']:>12.2f}  {m['rating']:>8}  {m['seats']:>6}")
    print()


def calculate_bill(price_per_ticket, quantity):
    subtotal = price_per_ticket * quantity
    gst = subtotal * GST_RATE
    total = subtotal + gst
    # round to 2 decimal places, consistent with currency
    subtotal = round(subtotal, 2)
    gst = round(gst, 2)
    total = round(total, 2)
    return subtotal, gst, total


def find_movie(movies, movie_id):
    for m in movies:
        if m["id"] == movie_id:
            return m
    return None


def book_tickets(movies):
    show_movies(movies)
    try:
        movie_id = int(input("Enter the movie ID you want to book: ").strip())
    except ValueError:
        print("Invalid input. Please enter a number for movie ID.")
        return

    movie = find_movie(movies, movie_id)
    if not movie:
        print("Movie not found. Please choose a valid ID.")
        return

    try:
        qty = int(input("Number of tickets (max 6 per transaction): ").strip())
    except ValueError:
        print("Invalid ticket count. Please enter a number.")
        return

    if qty <= 0:
        print("You must book at least 1 ticket.")
        return

    if qty > MAX_TICKETS_PER_TX:
        print(f"Cannot book more than {MAX_TICKETS_PER_TX} tickets in a single transaction.")
        return

    if qty > movie["seats"]:
        print(f"Requested {qty} tickets but only {movie['seats']} seats are available.")
        return

    # Age check for 'A' rated movie
    user_age = None
    if movie["rating"].upper() == "A":
        try:
            user_age = int(input("This movie is rated 'A' (Adults only). Enter your age: ").strip())
        except ValueError:
            print("Invalid age entered.")
            return
        if user_age < 18:
            print("You must be at least 18 years old to watch 'A' rated movies. Booking rejected.")
            return

    # optional: collect customer name
    customer_name = input("Enter customer name (optional): ").strip() or "Guest"

    subtotal, gst, total = calculate_bill(movie["price"], qty)

    # Confirm booking
    print("\nBooking Summary:\n")
    print(f"Movie: {movie['name']}")
    print(f"Tickets: {qty}")
    print(f"Price per ticket: ₹{movie['price']:.2f}")
    print(f"Subtotal: ₹{subtotal:.2f}")
    print(f"GST (5%): ₹{gst:.2f}")
    print(f"Total: ₹{total:.2f}")

    confirm = input("Confirm booking? (y/n): ").strip().lower()
    if confirm != "y":
        print("Booking cancelled.")
        return

    # Update seats
    movie["seats"] -= qty
    save_movies(movies)

    booking = {
        "timestamp": datetime.now().isoformat(),
        "customer": customer_name,
        "movie_id": movie["id"],
        "movie_name": movie["name"],
        "rating": movie["rating"],
        "tickets": qty,
        "price_per_ticket": movie["price"],
        "subtotal": subtotal,
        "gst": gst,
        "total": total,
    }

    save_booking(booking)

    # Print a neat ticket confirmation
    print_ticket(booking, movie["seats"]) 


def print_ticket(booking, seats_left):
    lines = []
    lines.append("\n" + "=" * 40)
    lines.append("\t	TICKET CONFIRMATION")
    lines.append("=" * 40)
    lines.append(f"Customer: {booking['customer']}")
    lines.append(f"Movie: {booking['movie_name']} ({booking['rating']})")
    lines.append(f"Tickets: {booking['tickets']}")
    lines.append(f"Price/ticket: ₹{booking['price_per_ticket']:.2f}")
    lines.append(f"Subtotal: ₹{booking['subtotal']:.2f}")
    lines.append(f"GST (5%): ₹{booking['gst']:.2f}")
    lines.append(f"Total Paid: ₹{booking['total']:.2f}")
    lines.append(f"Seats remaining for this movie: {seats_left}")
    lines.append(f"Booking time: {booking['timestamp']}")
    lines.append("=" * 40 + "\n")

    print("\n".join(lines))


def view_bookings():
    bookings = load_bookings()
    if not bookings:
        print("\nNo bookings found.")
        return
    print(f"\nPrevious bookings (most recent first) - total {len(bookings)}\n")
    for b in reversed(bookings[-20:]):  # show last 20 bookings
        print(f"{b['timestamp']}: {b['customer']} booked {b['tickets']} x {b['movie_name']} for ₹{b['total']:.2f}")
    print()


def reset_data_prompt():
    print("\nDanger: this will reset movies seats to default and delete bookings file if confirmed.")
    c = input("Type 'RESET' to perform reset, anything else to cancel: ").strip()
    if c == "RESET":
        if os.path.exists(MOVIES_STATE_FILE):
            os.remove(MOVIES_STATE_FILE)
        if os.path.exists(BOOKINGS_FILE):
            os.remove(BOOKINGS_FILE)
        # reinitialize
        save_movies(DEFAULT_MOVIES.copy())
        print("Data reset to defaults.")
    else:
        print("Reset cancelled.")


# --------------------------- Main app flow ---------------------------

def movie_system():
    movies = load_movies()
    while True:
        print("\n========== Movie Ticket Booking System ==========")
        print("1. Show available movies")
        print("2. Book tickets")
        print("3. View previous bookings")
        print("4. Reset data (danger)")
        print("5. Exit")

        choice = input("Choose an option (1-5): ").strip()
        if choice == "1":
            show_movies(movies)
        elif choice == "2":
            # reload movies from disk in case external change
            movies = load_movies()
            book_tickets(movies)
        elif choice == "3":
            view_bookings()
        elif choice == "4":
            reset_data_prompt()
            movies = load_movies()
        elif choice == "5":
            print("Goodbye!")
            break
        else:
            print("Invalid choice. Please select a number from 1 to 5.")


if __name__ == "__main__":
    try:
        movie_system()
    except KeyboardInterrupt:
        print("\nInterrupted. Exiting gracefully.")


SyntaxError: unterminated string literal (detected at line 68) (2429237979.py, line 68)